In [3]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


In [4]:
from bs4 import BeautifulSoup

In [5]:
import bs4

In [6]:
import json

# I. Get a list of document titles

In [17]:
mypath = "edgedriver_win64\\msedgedriver.exe"
#s = Service(mypath)
driver = webdriver.Edge(executable_path= mypath)

In [18]:
url1 ="https://advance.lexis.com/search/?pdmfid=1516831&crid=f1531e5b-e2a5-448c-bdeb-34e123fd4485&pdsearchtype=SearchBox&pdtypeofsearch=searchboxclick&pdstartin=&pdsearchterms=legal+case&pdtimeline=Previous+2+Years+%7C+p2y&pdpsf=&pdquerytemplateid=&pdsf=&ecomp=3bJgkgk&prid=387b52f1-8a0e-4526-b00e-161688824ebf"
driver.get(url1)

In [19]:
#get title of documents

def get_doc_title():
    time.sleep(5)
    
    title_elements = driver.find_elements(By.XPATH,"//ol[@class='nexisuni-result-list']/li[{n}]/div/h2/a".format(n = i*2+2))
    title_texts = []
    for t in title_elements:
        title_text = t.text
        title_texts.append(title_text)
    return title_texts

In [20]:
list_of_list_title = []
for k in range(250):
    for i in range(10):
        doc = driver.find_element(By.XPATH,"//ol[@class='nexisuni-result-list']/li[{n}]/div/div[2]/article/p".format(n = i*2+2))
        overview = doc.get_attribute("class")
        if overview == "overview min":
            list_of_list_title.append(get_doc_title())
    time.sleep(5)
    next_page = driver.find_element(By.XPATH,"//a[@class='icon la-TriangleRight action' and @href='#' and @data-action='nextpage']")
    next_page.click()
    
driver.close()

In [21]:
list_of_list_title

[['Midwest Sanitary Serv. v. Sandberg, 2022 IL 127327'],
 ['Young v. Blake, 2022 Ark. App. 378'],
 ['Fabec v. Frederick & Berler, LLC, 2022-Ohio-376'],
 ['Ortiz v. Olandapo-Jimoh, 2022 Tex. App. LEXIS 6487'],
 ['Royal Palm Props., LLC v. Pink Palm Props., LLC, 38 F.4th 1372'],
 ['McGilberry v. Ross, 2022 Miss. App. LEXIS 318'],
 ['Klevenhagen v. Hilburn, 2022 Tex. App. LEXIS 8370'],
 ['Gandy v. Williamson, 634 S.W.3d 214'],
 ['Brumfield v. Williamson, 634 S.W.3d 170'],
 ['Terry v. Bell, 2021-Ohio-4235'],
 ['Calderin v. Quartz Hill Mining, LLC, 317 So. 3d 243'],
 ['In re A.B., 2021-Ohio-4134'],
 ['Henry v. Ahern, 335 So. 3d 871'],
 ['Bressi v. Irwin, 2021-Ohio-2550'],
 ['In re A.B., 79 Cal. App. 5th 906'],
 ['Khalil v. Williams, 278 A.3d 859'],
 ['Winstead PC v. Moore, 633 S.W.3d 200'],
 ['Young v. Lippl, 202 Pa. Super. 56'],
 ['Lairy v. Chandler, 341 So. 3d 919'],
 ['In re E.E.D., 2022-Ohio-4014'],
 ['People v. Superior Court (Jones), 12 Cal. 5th 348'],
 ['Compher v. Whitfield, 2022 Te

In [22]:
document_titles = []

In [23]:
document_titles = [item for element in list_of_list_title for item in element]


In [24]:
len(document_titles)

1211

In [35]:
len(document_titles)

19

In [25]:
my_file = open('document_titles.txt', 'w', encoding="utf-8", newline="")

In [26]:
for t in document_titles:
    my_file.write(t)
    my_file.write('\n')
my_file.close()

# II. Get content of document from the above document title

In [27]:
def get_core_term():
    core_term_dict = {}
    core_terms = []
    all_elements = driver.find_element(By.XPATH,"//main[@role='main']")

    #all_elements = driver.find_element(By.XPATH,"//main[@id='4sk8k']")
    html_doc = all_elements.get_attribute('outerHTML')
    soup = BeautifulSoup(html_doc,"html.parser")
    child_lists = list(soup.select('.SS_LeftAlign')[0].children)
    for c in child_lists:
        if (c.text) == 'Core Terms':
            index_ = int(child_lists.index(c))
    coreterms = child_lists[index_+2]
    
    core_term_dict['Core Terms'] = coreterms
    return core_term_dict

In [29]:
def get_overview_doc():
    body_doc = driver.find_elements(By.XPATH,"//span[@class='SS_EditorialContent']/*")
    overview_dict = {}
    myKey = ''
    myValue = ''
    for i in range(len(body_doc)):
        tag_name = body_doc[i].tag_name
        if tag_name.startswith('span'):
            myKey = body_doc[i].text
        elif tag_name.startswith('p'):
            myValue = body_doc[i].text
        if myKey and myValue:
            overview_dict[myKey] = myValue
    return overview_dict


In [30]:
def get_body_doc():
    all_content_elements = driver.find_elements(By.XPATH,"//span[@class='SS_LeftAlign']/*")
    detail_dict = {}
    myKey = ''
    myValue = ''
    for i in range(len(all_content_elements)):
        attribute = all_content_elements[i].get_attribute('class')
        tag_name = all_content_elements[i].tag_name
        if tag_name.startswith('span') and attribute == 'SS_bf':
            if myKey:
                detail_dict[myKey] = myValue
                myKey = ''
                myValue = ''
            myKey = all_content_elements[i].text
        if tag_name.startswith('p') and myKey:
            myValue = myValue + all_content_elements[i].text
    detail_dict[myKey] = myValue
    return detail_dict

In [31]:
mypath = "edgedriver_win64\\msedgedriver.exe"
#s = Service(mypath)
driver = webdriver.Edge(executable_path= mypath)

In [32]:
url2 = 'https://advance-lexis-com.libproxy.library.unt.edu/search/?pdmfid=1516831&crid=cacd72d1-84b5-470a-8358-7b6f69c84643&pdsearchtype=SearchBox&pdtypeofsearch=searchboxclick&pdstartin=&pdsearchterms=Zive+v.+Sandberg%2C+610+S.W.3d+44&pdtimeline=Previous+2+Years+%7C+p2y&pdpsf=&pdquerytemplateid=&pdsf=&ecomp=bbJgkgk&prid=033a47fc-431f-4814-b8a3-ac5068c93b58'

In [33]:
driver.get(url2)

In [47]:
documents = []

In [56]:
for title in document_titles[100:]:
    document_dict = {}
    core_term_dict = {}
    overview_dict = {}
    body_doc_dict = {}
    document_dict['Title'] = title
    #time.sleep(5)
    #driver.find_element(By.XPATH,"//textarea[@aria-label='Enter Search Term' and @type ='text']").clear()
    driver.find_element(By.TAG_NAME,'textarea').clear()
    time.sleep(3)
    driver.find_element(By.TAG_NAME,'textarea').send_keys(title)
    time.sleep(3)
    button = driver.find_element(By.XPATH,"//button[@type='button' and @class = 'icon la-Search mainSearch' and @aria-label='Search']")
    button.click()
    try:
        core_term_dict = get_core_term()
        time.sleep(4)
        overview_dict = get_overview_doc()
        time.sleep(4)
        body_doc_dict = get_body_doc()
        for i in core_term_dict:
            document_dict[i] = core_term_dict[i]
        for i in overview_dict:
            document_dict[i] = overview_dict[i]
        for i in body_doc_dict:
            document_dict[i] = body_doc_dict[i] 
    except:
        continue
    with open('200_legal_case_documents.json','a') as outfile:
        json.dump(document_dict,outfile)
        outfile.close()    
    documents.append(document_dict)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"textarea"}
  (Session info: MicrosoftEdge=108.0.1462.42)


In [55]:
document_titles[99]

'Liberatore v. Castorani, 2021-Ohio-1684'

In [103]:
driver.close()

In [106]:
myfile = open('200_legal_case_documents.json','w')

json.dump(documents,myfile)

myfile.close()

In [116]:
f = open('legal_case_documents.json','r')
data = json.load(f)

In [119]:
f.close()